1. ## ⚙️ Setup: Mount Drive & Install Libraries

This first step will mount your Google Drive so the script can access your files and install the necessary transformers library from Hugging Face.

In [ ]:
from google.colab import drive
import os
import pandas as pd
import numpy as np
import glob
from PIL import Image
from transformers import pipeline

# Mount your Google Drive
# This will prompt you for authorization
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 2. 🗂️ Load Data & AI Model

Here, we'll define the paths to your files, read your main CSV, and load the openai/clip-vit-base-patch32 model into a pipeline.

Important: You must change the file paths in the ### --- CONFIGURATION --- ### section to match the locations of your files in your Google Drive.

In [ ]:
### --- CONFIGURATION --- ###
# ⚠️ UPDATE THESE PATHS to point to your files in Google Drive

# Path to your main CSV file
CSV_FILE_PATH = '/content/drive/My Drive/Work - Bisk College/Data/CSV_images/clean_everything.csv'

# Path to the FOLDER containing your images
IMAGE_FOLDER_PATH = '/content/drive/My Drive/Work - Bisk College/Data/Downloaded_Images'

# Path where you want to SAVE the new results CSV
OUTPUT_CSV_PATH = '/content/drive/My Drive/Work - Bisk College/Data/Sentiment_historical/Analysis_sentimental_openai_clip-vit-base-patch32_model.csv'

### --- END CONFIGURATION --- ###


# 1. Load the main CSV file
try:
    # Adding on_bad_lines='skip' to handle potential formatting issues in the CSV
    df = pd.read_csv(CSV_FILE_PATH, sep=';')
    # Ensure Index column is treated as the correct type for matching (e.g., integer)
    # If your indices are numbers, converting to int is safest.
    # If they can be strings, use str.
    df['Index'] = pd.to_numeric(df['Index'], errors='coerce').fillna(0).astype(int)
    print("Successfully loaded main CSV file.")
    print(df.head())
except FileNotFoundError:
    print(f"ERROR: Could not find the CSV file at {CSV_FILE_PATH}")
except Exception as e:
    print(f"An error occurred loading the CSV: {e}")

# 2. Load the AI Model (Zero-Shot Image Classification Pipeline)
# This will download the model, which might take a minute
print("Loading model 'openai/clip-vit-base-patch32'...")
try:
    # We use the "zero-shot-image-classification" pipeline
    # This allows us to classify an image against arbitrary text labels
    classifier = pipeline("zero-shot-image-classification",
                          model="openai/clip-vit-base-patch32")
    print("Model loaded successfully.")
except Exception as e:
    print(f"Error loading model: {e}")

Successfully loaded main CSV file.
         Date  Newspaper                                          Image_URL  \
0  01/01/2016  USA Today  https://web.archive.org/web/20160101193049im_/...   
1  01/01/2016  USA Today  https://web.archive.org/web/20160101193049im_/...   
2  01/01/2016  USA Today  https://web.archive.org/web/20160101193049im_/...   
3  01/01/2016  USA Today  https://web.archive.org/web/20160101193049im_/...   
4  01/01/2016  USA Today  https://web.archive.org/web/20160101193049im_/...   

                                             Caption  Index  
0                                          USA TODAY      1  
1                                          USA TODAY      2  
2  Singer Natalie Cole performs in New York in 2013.      3  
3  Natalie Cole performs at Apollo Theater Spring...      4  
4  You're not going to start your New Year's reso...      5  
Loading model 'openai/clip-vit-base-patch32'...


Device set to use cpu


Model loaded successfully.


## 3. 🖼️ Process Images & Perform Analysis

This is the main part of the script. It will:

1.   Find all images in your folder.
2.   Sort them by their numeric name (e.g., 1, 2, 5, 10...).
3.   Loop through each image.
4.   Find the matching row in your CSV using the image's name as the Index.
5.   Perform "multimodal" analysis if a caption exists (using the caption to create specific labels).
6.   Perform "image-only" analysis if no caption is found (using generic sentiment labels).
7.   Store all the results.



In [ ]:
# A list to store all our results dictionaries
results_list = []

# Define the base sentiment labels
BASE_SENTIMENTS = ["positive", "negative", "neutral"]

# 1. Get all image paths from the folder
# Using glob.glob(..., recursive=True) could find files in subfolders if needed
# For now, '*' finds all files in the top-level folder
image_paths = glob.glob(os.path.join(IMAGE_FOLDER_PATH, '*'))

# 2. Sort image paths numerically by their filename
sorted_images = []
for path in image_paths:
    try:
        # Get filename without extension (e.g., '1', '5')
        filename = os.path.basename(path)
        index_str = os.path.splitext(filename)[0]
        # Convert to integer for correct sorting
        image_index = int(index_str)
        sorted_images.append((image_index, path))
    except ValueError:
        # This handles cases where a file in the folder
        # doesn't have a numeric name (e.g., 'notes.txt')
        print(f"Skipping non-numeric filename: {filename}")

# Sort the list based on the integer index (the first item in our tuple)
sorted_images.sort()
print(f"\nFound and sorted {len(sorted_images)} images to process.")

# 3. Loop through each sorted image
for image_index, image_path in sorted_images:

    # This try/except block handles errors for a *single image*
    # (e.g., corrupt file, model error) so the script doesn't crash.
    # This addresses your "IMPORTANT" point 2 about other errors.
    try:
        print(f"Processing image index: {image_index}...")

        # Open the image
        image = Image.open(image_path)

        # Find the matching row in the DataFrame
        row_data = df[df['Index'] == image_index]

        # Initialize variables
        caption = np.nan
        date = 'Unknown'

        if not row_data.empty:
            # Get date (if row exists)
            date = row_data.iloc[0]['Date']
            # Check if a caption exists and is not empty/NaN
            if pd.notna(row_data.iloc[0]['Caption']):
                caption = row_data.iloc[0]['Caption']

        # --- Add the condition to skip processing if caption is "USA TODAY" ---
        if pd.notna(caption) and caption.strip().upper() == "USA TODAY":
            print(f"  Skipping image {image_index} as caption is 'USA TODAY'.")
            continue # Skip the rest of the loop for this image and go to the next one

        # --- Perform Analysis ---

        if pd.notna(caption):
            # 5. MULTIMODAL SENTIMENT ANALYSIS
            # We use the caption to create specific, contextual labels
            # This guides the model to evaluate the image *in the context* of the caption

            # Create a mapping: simple label -> complex label
            labels_map = {
                s: f"a photo about '{caption}' with {s} sentiment"
                for s in BASE_SENTIMENTS
            }

            # Run classifier with the complex labels
            raw_results = classifier(image, candidate_labels=list(labels_map.values()))

            # Get the top prediction (which is a complex label)
            predicted_label_text = raw_results[0]['label']

            # Find the simple sentiment (key) that corresponds to the predicted label (value)
            predicted_sentiment = [
                key for key, value in labels_map.items()
                if value == predicted_label_text
            ][0] # Get the first (and only) match

        else:
            # 6. IMAGE-ONLY SENTIMENT ANALYSIS
            # No caption exists, so we use generic sentiment labels

            labels = [f"{s} sentiment" for s in BASE_SENTIMENTS]
            raw_results = classifier(image, candidate_labels=labels)

            # Extract the simple sentiment (e.g., "positive" from "positive sentiment")
            predicted_sentiment = raw_results[0]['label'].split(' ')[0]

        # Get the confidence score
        confidence = raw_results[0]['score']

        # 7. Store the result
        results_list.append({
            'date': date,
            'image_index': image_index,
            'caption': caption,
            'Predicted_sentiment': predicted_sentiment,
            'confidence': confidence
        })

    except Exception as e:
        print(f"--- ERROR processing image {image_index} ({image_path}): {e}")
        print("--- Skipping this image and continuing.")

print("\nProcessing complete.")


Found and sorted 15110 images to process.
Processing image index: 1...
  Skipping image 1 as caption is 'USA TODAY'.
Processing image index: 2...
  Skipping image 2 as caption is 'USA TODAY'.
Processing image index: 3...
Processing image index: 4...
Processing image index: 5...
Processing image index: 6...
Processing image index: 7...
Processing image index: 8...
Processing image index: 9...
Processing image index: 10...
Processing image index: 11...
  Skipping image 11 as caption is 'USA TODAY'.
Processing image index: 12...
  Skipping image 12 as caption is 'USA TODAY'.
Processing image index: 13...
Processing image index: 14...
Processing image index: 15...
Processing image index: 16...
Processing image index: 17...
Processing image index: 18...
Processing image index: 19...
Processing image index: 20...
Processing image index: 21...
  Skipping image 21 as caption is 'USA TODAY'.
Processing image index: 22...
  Skipping image 22 as caption is 'USA TODAY'.
Processing image index: 23

Token indices sequence length is longer than the specified maximum sequence length for this model (78 > 77). Running this sequence through the model will result in indexing errors


Streaming output truncated to the last 5000 lines.
Processing image index: 11386...
Processing image index: 11387...
Processing image index: 11388...
Processing image index: 11389...
Processing image index: 11390...
Processing image index: 11391...
Processing image index: 11392...
Processing image index: 11393...
Processing image index: 11394...
Processing image index: 11395...
Processing image index: 11396...
Processing image index: 11397...
Processing image index: 11398...
Processing image index: 11400...
Processing image index: 11402...
Processing image index: 11403...
Processing image index: 11404...
Processing image index: 11405...
Processing image index: 11406...
Processing image index: 11407...
Processing image index: 11408...
Processing image index: 11409...
Processing image index: 11410...
Processing image index: 11411...
Processing image index: 11412...
Processing image index: 11413...
Processing image index: 11414...
Processing image index: 11415...
Processing image index: 1

The channel dimension is ambiguous. Got image shape (1, 1, 3). Assuming channels are the first dimension. Use the [input_data_format](https://huggingface.co/docs/transformers/main/internal/image_processing_utils#transformers.image_transforms.rescale.input_data_format) parameter to assign the channel dimension.
The channel dimension is ambiguous. Got image shape (1, 1, 3). Assuming channels are the first dimension. Use the [input_data_format](https://huggingface.co/docs/transformers/main/internal/image_processing_utils#transformers.image_transforms.rescale.input_data_format) parameter to assign the channel dimension.
The channel dimension is ambiguous. Got image shape (1, 1, 3). Assuming channels are the first dimension. Use the [input_data_format](https://huggingface.co/docs/transformers/main/internal/image_processing_utils#transformers.image_transforms.rescale.input_data_format) parameter to assign the channel dimension.
The channel dimension is ambiguous. Got image shape (1, 1, 3). A

Processing image index: 20800...
--- ERROR processing image 20800 (/content/drive/My Drive/Work - Bisk College/Data/Downloaded_Images/20800.gif): mean must have 1 elements if it is an iterable, got 3
--- Skipping this image and continuing.
Processing image index: 20892...
--- ERROR processing image 20892 (/content/drive/My Drive/Work - Bisk College/Data/Downloaded_Images/20892.gif): mean must have 1 elements if it is an iterable, got 3
--- Skipping this image and continuing.
Processing image index: 21099...
--- ERROR processing image 21099 (/content/drive/My Drive/Work - Bisk College/Data/Downloaded_Images/21099.gif): mean must have 1 elements if it is an iterable, got 3
--- Skipping this image and continuing.
Processing image index: 21230...
--- ERROR processing image 21230 (/content/drive/My Drive/Work - Bisk College/Data/Downloaded_Images/21230.gif): mean must have 1 elements if it is an iterable, got 3
--- Skipping this image and continuing.
Processing image index: 21416...
--- ERR

## 4. 💾 Save Results

Finally, this cell converts the list of results into a pandas DataFrame and saves it to the output CSV file you specified.

In [ ]:
# Convert the list of dictionaries into a DataFrame
results_df = pd.DataFrame(results_list)

# Display the first few rows of the results
print("\n--- Results ---")
print(results_df.head())

# Save the DataFrame to the output CSV file
try:
    results_df.to_csv(OUTPUT_CSV_PATH, index=False)
    print(f"\n✅ Successfully saved results to: {OUTPUT_CSV_PATH}")
except Exception as e:
    print(f"\n❌ ERROR: Could not save results. {e}")


--- Results ---
         date  image_index                                            caption  \
0  01/01/2016            3  Singer Natalie Cole performs in New York in 2013.   
1  01/01/2016            4  Natalie Cole performs at Apollo Theater Spring...   
2  01/01/2016            5  You're not going to start your New Year's reso...   
3  01/01/2016            6  Authorities apprehended the Texas teen and his...   
4  01/01/2016            7                                                NaN   

  Predicted_sentiment  confidence  
0             neutral    0.526676  
1             neutral    0.387742  
2            negative    0.540173  
3            negative    0.441595  
4             neutral    0.359617  

✅ Successfully saved results to: /content/drive/My Drive/Work - Bisk College/Data/Sentiment_historical/Analysis_sentimental_openai_clip-vit-base-patch32_model.csv
